https://stackoverflow.com/questions/3588776/how-is-eq-handled-in-python-and-in-what-order
https://www.tutorialsteacher.com/python/magic-methods-in-python
https://stackoverflow.com/questions/2988017/string-comparison-in-python-is-vs

is is used for identity comparison, while == is used for equality comparison. Since what you care about is equality (the two strings should contain the same characters), in this case the is operator is simply wrong and you should be using == instead.

The reason is works interactively is that (most) string literals are interned by default. From Wikipedia:

Interned strings speed up string comparisons, which are sometimes a performance bottleneck in applications (such as compilers and dynamic programming language runtimes) that rely heavily on hash tables with string keys. Without interning, checking that two different strings are equal involves examining every character of both strings. This is slow for several reasons: it is inherently O(n) in the length of the strings; it typically requires reads from several regions of memory, which take time; and the reads fills up the processor cache, meaning there is less cache available for other needs. With interned strings, a simple object identity test suffices after the original intern operation; this is typically implemented as a pointer equality test, normally just a single machine instruction with no memory reference at all.

So, when you have two string literals (words that are literally typed into your program source code, surrounded by quotation marks) in your program that have the same value, the Python compiler will automatically intern the strings, making them both stored at the same memory location. (Note that this doesn't always happen, and the rules for when this happens are quite convoluted, so please don't rely on this behavior in production code!)

Since in your interactive session both strings are actually stored in the same memory location, they have the same identity, so the is operator works as expected. But if you construct a string by some other method (even if that string contains exactly the same characters), then the string may be equal, but it is not the same string -- that is, it has a different identity, because it is stored in a different place in memory.

In [9]:
a = "Hello"
b = "Hello"
print(a is b)
print(a.__eq__(b))
print(a==b)
print(a==None)
print(id(a))
print(id(b))

True
True
True
False
1896138313944
1896138313944


In [8]:
#If you run this as a script,
# a is b will actually probably be true, but if 
# you run it in a REPL, it won't be. That's because of
# how the interpreter optimizes this Python file.
a = "This is a string"
b = "This is a string"

print("a == b:", a == b)
print("a is b:", a is b)

# Neither a == c or a is c
c = "This is a different string" 

print("a == c:", a == c)
print("a is c:", a is c)

# a_list and b_list point to the same object,
# so the is condition is satisfied.
a_list = [1, 2, 3]
b_list = a_list

print("a_list is b_list:", a_list is b_list)

# appending to one appends to both
# because both point to the same object
a_list.append(4)
print("a_list:", a_list)
print("b_list:", b_list)

# Interned objects will point to the same
# underlying object
a_int = 4
b_int = 4
print("a_int is b_int:", a_int is b_int)
print("a_int == b_int:", a_int == b_int)

a == b: True
a is b: False
a == c: False
a is c: False
a_list is b_list: True
a_list: [1, 2, 3, 4]
b_list: [1, 2, 3, 4]
a_int is b_int: True
a_int == b_int: True


In [10]:
class SillyString(str):
    # This method gets called when using == on the object
    def __eq__(self, other):
        # Uncomment below if you want verbose output
        # print(f'comparing {self} to {other}')
        # Return True if self and other have the same length
        return len(self) == len(other)

# Normal strings compare by character
print("'hello world' == 'world hello':", 'hello world' == 'world hello')

# But SillyStrings compare by length alone
# by calling the __eq__() method
print("SillyString('hello world') == 'world hello':", SillyString('hello world') == 'world hello')

# Remember that == calls the left-hand operand's __eq__() method 
# Unless the right-hand operand is a subclass of the left
print("'hello world' == SillyString('world hello'):", 'hello world' == SillyString('world hello'))

# But the SillyString method isn't very well defined
# So it only compares with length and does no type-checking
print("SillyString('hello world') == {list of 11 elements}:", \
        SillyString('hello world') == [0 for i in range(11)])

# Comparing to None generates an error because we don't check
# for whether either object is None
# and None has no len() atribute
print("SillyString('hello_world') == None:", SillyString('hello_world') == None)

# Comment the above out and use is instead
print("SillyString('hello_world') is None:", SillyString('hello_world') is None)



'hello world' == 'world hello': False
SillyString('hello world') == 'world hello': True
'hello world' == SillyString('world hello'): True
SillyString('hello world') == {list of 11 elements}: True


TypeError: object of type 'NoneType' has no len()

In [12]:
from sys import intern
a = str("This is a string")
b = str("This is a string")
print("a == b:", a == b)
print("a is b:", a is b)
# now they will point to same location
a= intern(a)
b= intern(b)
print("a == b:", a == b)
print("a is b:", a is b)

a == b: True
a is b: False
a == b: True
a is b: True
